In [29]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Karanjudyani/machine-learning-mlops-portfolio.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="karanjudyani"
os.environ["MLFLOW_TRACKING_PASSWORD"]="51f3e7ff62e078c30c08b11a0d6e025c8cf5f9e6"

In [30]:
import os
%pwd

'd:\\ml-mlops-portfolio\\wine_quality_mlops'

In [31]:
# os.chdir("../")
# %pwd

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [33]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Karanjudyani/machine-learning-mlops-portfolio.mlflow"


        )
        return model_evaluation_config


In [35]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [36]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # DagsHub doesn't support sklearn log_model or registry
            # So we save model locally and log manually
            local_model_path = "model.pkl"
            joblib.dump(model, local_model_path)
            mlflow.log_artifact(local_model_path)

    


In [37]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-07 13:19:34,406: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-07 13:19:34,406: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-07 13:19:34,406: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-07 13:19:34,416: INFO: common: created directory at: artifacts]
[2025-08-07 13:19:34,418: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-07 13:19:34,582: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run casual-fox-567 at: https://dagshub.com/Karanjudyani/machine-learning-mlops-portfolio.mlflow/#/experiments/0/runs/93f9387872c146ffbb49adf3363c42e6
🧪 View experiment at: https://dagshub.com/Karanjudyani/machine-learning-mlops-portfolio.mlflow/#/experiments/0
